<h3><center>Reconhecedor de Dígitos numéricos</center></h3>
<h4>Rede Neural Simples</h4>
<li>Projeto de estudo de rede neurais multicamadas (duas).</li>
<li>O desafio apresentado está no link do kaggle, que é uma base de dados de arquivos abertos para estudo de Data Science e pesquisas estatisticas</li>
<li>A proposta do desafio consiste em implementar uma rede neural que seja capaz de identificar digitos escritos a mão em imagens de resulução de 28X28 pixel que são baixados num formato de array com o labe, valores em tons de cinza que variam de 0 (branco) a 1 (preto) compondo a estrutura da matriz que forma a imagem.
<br/><br/>
<center><img src="utils/images/grey_scale_matrix.png" style="width: 40%; height:40%"></center>



<h4>Import de Bibliotecas utilizadas<h4>

In [1]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

<h4>Importação da base de dados dos dados</h4>
<li>Fonte dos dados de estudo: <pre>https://www.kaggle.com/c/digit-recognizer/data?select=train.csv </pre>

In [2]:
treino = pd.read_csv('utils/data/train.csv')
teste = pd.read_csv('utils/data/test.csv')

<h4>Tratamento dos dados</h4>
<br/><br/>
<center><img src="utils/images/matrices_nn.png" style="width: 40%; height:40%"></center>

In [3]:
treino = np.array(treino)
teste = np.array(teste)

np.random.shuffle(treino)

dados_teste = teste.T
dados_treino = treino.T

Y_treino = dados_treino[0]

X_teste = dados_teste[:]
X_treino = dados_treino[1:]

X_teste = X_teste / 255.
X_treino = X_treino / 255.

_, c_treino = X_treino.shape
_, c_teste = X_teste.shape

<h4>Visualização de Imagens do Dataset</h4>
<li>Função para mostrar imagens de amostras aleatórias do dataset</li>

In [4]:
def mostrar_imagens(indice):
    imagem = X_teste[:, indice, None]
    imagem = imagem.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(imagem, interpolation='nearest')
    plt.show()

<h4>Inicialização da Rede Neural</h4>
<li>Inicia os pesos e vieses aleatoriamente para a camada de entrada e a camada oculta</li>

In [5]:
def montar_rede():
    W1 = np.random.rand(10, 784) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, W2, b1, b2

<h4>Funções de Ativação</h4>
<li>Inicia os pesos e vieses aleatoriamente para a camada de entrada e a camada oculta</li>
<li>A função de ativação ajuda a introduzir não-linearidade no modelo, permitindo que ele aprenda padrões complexos nos dados.</h4>

In [6]:
def ReLU(Z):
    return np.maximum(0, Z)


def softmax(Z):
    e = np.exp(Z)
    return e / e.sum(axis=0)

 <h4>Propagação Feedforward</h4>
<li>Propagação da entrada até a saída, retornando os valores calculados nas camada</li>

In [7]:
def forward_prop(W1, W2, b1, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

<h4>Função de previsão e Precisão</h4>
<li>A função de previsão recebe como entrada o vetor de saída da rede neural, após a aplicação da função de ativação softmax. Essa saída contém as probabilidades de cada classe.</li>
<li> função de precisão calcula a porcentagem de previsões corretas, comparando as previsões feitas pelo modelo com os rótulos corretos.</li>

In [8]:
# Retorna a posição com a maior probabilidade encontrada pela rede neural
def previsao(A2):
    return np.argmax(A2, axis=0)


# Calcula o percentual de acerto da rede neural
def precisao(previsao, Y):
    return np.sum(previsao == Y) / Y.size

<h4>Ajuste Aleatório dos Pesos</h4>
<li>Teste com inicialização aleatória dos pesos para identificar os melhores valores iniciais</li>

In [10]:
def randomizar(X, Y, alpha, iteracoes):
    melhor_p = 0
    melhor_W1 = 0
    melhor_W2 = 0
    melhor_b1 = 0
    melhor_b2 = 0
    W1, W2, b1, b2 = montar_rede()
    for i in range(iteracoes):
        Z1, A1, Z2, A2 = forward_prop(W1, W2, b1, b2, X)
        previsoes = previsao(A2)
        p = precisao(previsoes, Y)
        if p > melhor_p:
            melhor_p = p
            melhor_W1 = W1
            melhor_W2 = W2
            melhor_b1 = b1
            melhor_b2 = b2
        W1, W2, b1, b2 = montar_rede()
        if i % 100 == 0:
            print("Iteração:", i)
            print("Precisão:", str(round(melhor_p * 100, 2)) + "%")
    return melhor_W1, melhor_W2, melhor_b1, melhor_b2


W1, W2, b1, b2 = randomizar(X_treino, Y_treino, 0.1, 1000)

Iteração: 0
Precisão: 15.3%
Iteração: 100
Precisão: 16.81%
Iteração: 200
Precisão: 16.81%
Iteração: 300
Precisão: 16.81%
Iteração: 400
Precisão: 18.39%
Iteração: 500
Precisão: 18.97%
Iteração: 600
Precisão: 18.97%
Iteração: 700
Precisão: 18.97%
Iteração: 800
Precisão: 18.97%
Iteração: 900
Precisão: 18.97%


<h4>Função Resposta</h4>
<li>É um array de "booleanos" que armazena as posições corretas com 1 e incorretas com 0.</li>

In [13]:
def respostas(Y):
    resp = np.zeros((Y.size, Y.max() + 1))
    resp[np.arange(Y.size), Y] = 1
    resp = resp.T
    return resp

In [12]:
def ReLU_deriv(Z):
    return Z > 0

<h4>Back Propagation<h4>

In [15]:
def back_prop(Z1, A1, Z2, A2, W2, X, Y):
    resp_Y = respostas(Y)
    dZ2 = A2 - resp_Y
    dW2 = 1 / c_treino * dZ2.dot(A1.T)
    db2 = 1 / c_treino * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / c_treino * dZ1.dot(X.T)
    db1 = 1 / c_treino * np.sum(dZ1)
    return db2, db1, dW1, dW2

<h4>Atualização dos pesos</h4>
<li>Atualização dos pesos atraves do gradiente descendente.</li>

In [14]:
def atualiza_pesos(W1, W2, b1, b2, dW1, dW2, db1, db2, alpha):
    W1 = W1 - alpha * dW1
    W2 = W2 - alpha * dW2
    b1 = b1 - alpha * db1
    b2 = b2 - alpha * db2
    return W1, W2, b1, b2

<h4>Gradiente Descendente</h4>
<br/><br/>
<center><img src="utils/images/gadrient_descendent.png" style="width: 50%; height:50%"></center>

In [16]:
def gradiente(X, Y, alpha, iteracoes):
    W1, W2, b1, b2 = montar_rede()
    for i in range(iteracoes):
        Z1, A1, Z2, A2 = forward_prop(W1, W2, b1, b2, X)
        db2, db1, dW1, dW2 = back_prop(Z1, A1, Z2, A2, W2, X, Y)
        W1, W2, b1, b2 = atualiza_pesos(W1, W2, b1, b2, dW1, dW2, db1, db2, alpha)
        if i % 100 == 0:
            print("Iterações:", i)
            previsoes = previsao(A2)
            print("Precisão:", str(round(precisao(previsoes, Y) * 100, 2)) + "%")
    return W1, W2, b1, b2


W1, W2, b1, b2 = gradiente(X_treino, Y_treino, 0.1, 1000)

Iterações: 0
Precisão: 10.47%
Iterações: 100
Precisão: 62.59%
Iterações: 200
Precisão: 76.9%
Iterações: 300
Precisão: 81.04%
Iterações: 400
Precisão: 83.25%
Iterações: 500
Precisão: 84.6%
Iterações: 600
Precisão: 85.4%
Iterações: 700
Precisão: 86.12%
Iterações: 800
Precisão: 86.69%
Iterações: 900
Precisão: 87.19%
